### KRX 데이터정보시스템
- 매일 주식거래 내용

['일자', '종가', '등락률', '시가', '고가', '저가', '거래대금'] 등 수집

In [1]:
from selenium import webdriver as wd
from selenium.webdriver import ActionChains # scroll down 사용하기 위하여 선서
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as bs

import datetime, time
from datetime import date

import pandas as pd
import numpy as np
import requests
import time
import os, sys, io

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
import sys
print(sys.version)
print(pd.__version__)

3.13.1 (tags/v3.13.1:0671451, Dec  3 2024, 19:06:28) [MSC v.1942 64 bit (AMD64)]
2.2.3


In [4]:
# module_path = os.path.abspath(os.path.join('.')) 
# sys.path.append(module_path+"\\data\\constant")

# from constants import COMPANY_CODE

from data.constant.constants import COMPANY_CODE

In [5]:
# service = Service(r"chromedriver/125/chromedriver.exe")
# options = wd.ChromeOptions()
# driver = wd.Chrome(service=service, options=options)

In [6]:
service = Service(r"chromedriver/131/chromedriver.exe")
# https://googlechromelabs.github.io/chrome-for-testing/ 로 가서 브라우저 버젼에 맞는 chromedriver download 후에 사용
options = wd.ChromeOptions()
driver = wd.Chrome(service=service, options=options)

In [7]:
os.getcwd()

'D:\\Jupyter\\factors_affecting_stock_price'

In [8]:
# https://googlechromelabs.github.io/chrome-for-testing/

In [9]:
'''
# !pip install chromedriver_autoinstaller

import chromedriver_autoinstaller

driver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path
time.sleep(1)

# driver = wd.Chrome(driver_name)
service = Service(ChromeDriverManager(version="114.0.5735.90").install())
options = wd.ChromeOptions()
driver = wd.Chrome(service=service, options=options)
'''

'\n# !pip install chromedriver_autoinstaller\n\nimport chromedriver_autoinstaller\n\ndriver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists\n                                      # and if it doesn\'t exist, download it automatically,\n                                      # then add chromedriver to path\ntime.sleep(1)\n\n# driver = wd.Chrome(driver_name)\nservice = Service(ChromeDriverManager(version="114.0.5735.90").install())\noptions = wd.ChromeOptions()\ndriver = wd.Chrome(service=service, options=options)\n'

In [10]:
def get_data_company(com_name):

    com_ticker = com_name[:6]
    # 회사이름 입력 Q 버튼
    css_name = '#btnisuCd_finder_stkisu0_0'
    element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_name)))
    driver.find_element(By.CSS_SELECTOR, css_name).click()
    time.sleep(2)

    # pop up된 입력창에서 회사이름 입력
    id_name = 'searchText__finder_stkisu0_0'
    element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.ID, id_name)))
    driver.find_element(By.ID, id_name).clear()
    time.sleep(1)

    driver.find_element(By.ID, id_name).send_keys(com_name)
    time.sleep(2)

    # 검색 버튼 푸시
    css_name = '#searchBtn__finder_stkisu0_0'
    element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_name)))
    driver.find_element(By.CSS_SELECTOR, css_name).click()
    time.sleep(2)

    # 테이블에서 최종 선택
    css_sel = '#jsGrid__finder_stkisu0_0 > tbody > tr:nth-child(1) > td:nth-child(1)'

    element = WebDriverWait(driver, 60).until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, css_sel), com_ticker))
    # 위 라인은 pop up 창이 사라질 때까지 기다리게 해 줌
    driver.find_element(By.CSS_SELECTOR, css_sel).click()
    time.sleep(2)

    return

In [11]:
def set_date(start_date, end_date): # 일정 기간 데이터 취득
    # end_date를 먼저 입력하고 start date 입력. 반대로 하면 start date가 이전날짜로  reset되어짐
    driver.find_element(By.ID, 'endDd').clear()
    driver.find_element(By.ID, 'endDd').send_keys(end_date)
    time.sleep(1)
    
    driver.find_element(By.ID, 'strdDd').clear()
    driver.find_element(By.ID, 'strdDd').send_keys(start_date)
    time.sleep(1)

In [12]:
def get_data(start_date, end_date):
    
    column_name = ['date', 'close', 'change', 'close_cr', 'open', 'high', 'low', 
                  'vol', 'vol_amount','total_amount', 'total_counts' ]
    # ['일자', '종가', '대비', '등락률', '시가', '고가', '저가', '거래량', 
    #                                 '거래대금', '시가총액', '상장주식수']
 
    start_str = start_date.strftime('%Y-%m-%d')
    ed_str = end_date.strftime('%Y-%m-%d')
    set_date(start_str, ed_str)

    # 테이블  취득 버튼 클릭 (우상귀)
    id_name = 'jsSearchButton'
    element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.ID, id_name)))
    driver.find_element(By.ID, id_name).click()
    time.sleep(1)

    # df = pd.read_html(driver.page_source, 
    #                       attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]
    
    df = pd.read_html(io.StringIO(str(driver.page_source)), 
                          attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]
    
    df.columns = column_name
    df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y/%m/%d"))
    df_get = df[['date', 'open', 'high', 'low', 'close', 'close_cr', 'vol']]
    
    return df_get

In [13]:
def non_empty_index_df(df_input, start_date, end_date): # 토,일,공휴일등 거래가 없는 일자도 모두 포함
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    df_input.set_index('date', inplace=True)
    df_out = pd.DataFrame(columns = df_input.columns)
    df_out.insert(0, 'date', date_range_ts)
    df_out.set_index('date', inplace=True)
    df_out.update(df_input)
    df_out.reset_index(inplace=True)
    return df_out

In [14]:
def concat_df(df_o, df):
    df_o = pd.concat([df_o, df], ignore_index=True)
    df_o.drop_duplicates(subset=['date'], keep='last', inplace=True)
    df_o.sort_values(by=[df_o.columns[0]], inplace=True)
    df_o.index = np.arange(0, len(df_o))  # 일련 번호 오름차순으로 재 설정
    return df_o

In [15]:
# 개장일이 아닌 row는 삭제하기 위하여 개장일 데이터 load
base_data_directory = './data/base_data/stock_market_holydays/'
opening_days_kor = pd.read_pickle(base_data_directory+'opening_days_kor.pkl') # 한국 개장일 데이터 

# df_in : common data, opening_days : Series from oprning_days_xxx.pkl
def select_openingdays(df_in, opening_days):
    df_sel = df_in['date'].apply(lambda x: True if x.date() in list(opening_days) else False)
    df_out = df_in[df_sel].reset_index(drop=True)
    return df_out

In [16]:
# driver.set_window_position(-10000,0) # hide windows
main_url = 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020103'
# 개별종목 시세추이 data-menu-id: MDC0201020103
driver.get(main_url)
time.sleep(1)

In [17]:
# driver.maximize_window()

In [18]:
# 백만원 단위 표시 선정
# css_sel = '#MDCSTAT017_FORM > div.CI-MDI-UNIT-WRAP > div > p:nth-child(2) \
#            > select.CI-MDI-UNIT-MONEY > option:nth-child(3)'
# driver.find_element(By.CSS_SELECTOR, css_sel).click()
# 위 코드를 아래와 같이 두 개로 분리시켜야만 동작함.

css_sel_1 = '#MDCSTAT017_FORM > div.CI-MDI-UNIT-WRAP > div > p:nth-child(2) \
           > select.CI-MDI-UNIT-MONEY'
css_opt = 'option:nth-child(3)'
sel_1 = driver.find_element(By.CSS_SELECTOR, css_sel_1) # 우선 
sel_1.find_element(By.CSS_SELECTOR, css_opt).click()
time.sleep(1)

In [19]:
code = COMPANY_CODE

# code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
#                  '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec'], '000660': ['SK하이닉스', 'skhynix']}
# code = {'302440': ['SK바이오사이언스', 'skbio']}

In [20]:
pkl_directory = 'data/company_pkl/historical_data/'
modi_directory = 'data/company_pkl/'
modification_time = pd.read_pickle(modi_directory + 'modification_time_company_his.pkl')
pkl_name= 'sec_historical.pkl'

In [21]:
df_o = pd.read_pickle(pkl_directory + pkl_name)
start_date = df_o['date'].iloc[len(df_o)-1]
start_date

Timestamp('2025-01-09 00:00:00')

In [22]:
pkl_directory = 'data/company_pkl/historical_data/'
modi_directory = 'data/company_pkl/'
modification_time = pd.read_pickle(modi_directory + 'modification_time_company_his.pkl')
total = len(code)

for i, (key, val) in enumerate(code.items()):
    com_name = "/".join([key, val[0]])
    pkl_name= '{}_historical.pkl'.format(val[1])
    try :
        df_o = pd.read_pickle(pkl_directory + pkl_name)
        start_date = df_o['date'].iloc[len(df_o)-1]
    except :
        start_date = datetime.date(2022, 1, 1)   # 데이터 취득 시작 일자     

#     start_date = datetime.date(2023, 5, 11)  # 데이터 취득 오류시 일시 사용
        
    end_date = datetime.date.today()
    get_data_company(com_name)
    df_get = get_data(start_date, end_date)
    df_out = non_empty_index_df(df_get, start_date, end_date)
#   NaN to null string. cause NaN does not replace original values in the original datafram    
    try :
        df_o = concat_df(df_o, df_out) # append df to original df
    except :
        df_o = df_out.copy()
        
    # 개장일이 아닌 row는 삭제
    df_o = select_openingdays(df_o, opening_days_kor)
    
    df_o.replace(np.nan, '', inplace=True)
    df_o.to_pickle(pkl_directory+pkl_name)
    df_o.to_csv(pkl_directory+pkl_name.replace('pkl','csv'))
    ## modification_time.loc[pkl_name][0] = datetime.datetime.now()
    modification_time.loc[0, pkl_name] = datetime.datetime.now() 
    
    print(com_name, f'{i+1}/{total}', end=', ') # 진행상황 확인용
    
modification_time.to_pickle(modi_directory+'modification_time_company_his.pkl')
modification_time.to_csv(modi_directory+'modification_time_company_his.csv')

005930/삼성전자 1/52, 373220/LG에너지솔루션 2/52, 000660/SK하이닉스 3/52, 207940/삼성바이오로직스 4/52, 006400/삼성SDI 5/52, 051910/LG화학 6/52, 005935/삼성전자우 7/52, 005380/현대차 8/52, 035420/NAVER 9/52, 000270/기아 10/52, 035720/카카오 11/52, 005490/POSCO홀딩스 12/52, 105560/KB금융 13/52, 028260/삼성물산 14/52, 068270/셀트리온 15/52, 012330/현대모비스 16/52, 055550/신한지주 17/52, 066570/LG전자 18/52, 003670/포스코퓨처엠 19/52, 096770/SK이노베이션 20/52, 033780/KT&G 21/52, 030200/KT 22/52, 003550/LG 23/52, 034730/SK 24/52, 032830/삼성생명 25/52, 086790/하나금융지주 26/52, 009150/삼성전기 27/52, 015760/한국전력 28/52, 034020/두산에너빌리티 29/52, 010130/고려아연 30/52, 017670/SK텔레콤 31/52, 011200/HMM 32/52, 000810/삼성화재 33/52, 051900/LG생활건강 34/52, 010950/S-Oil 35/52, 259960/크래프톤 36/52, 018260/삼성에스디에스 37/52, 329180/현대중공업 38/52, 003490/대한항공 39/52, 036570/엔씨소프트 40/52, 009830/한화솔루션 41/52, 316140/우리금융지주 42/52, 090430/아모레퍼시픽 43/52, 011170/롯데케미칼 44/52, 024110/기업은행 45/52, 138040/메리츠금융지주 46/52, 377300/카카오페이 47/52, 011070/LG이노텍 48/52, 028050/삼성엔지니어링 49/52, 361610/SK아이이테크놀로지 50/52, 086280/현대글로비스

### 투자자별 현황
 - 거래 정보와 코딩 구성등 형태는 똑 같으나, empty_datetime index 내용과, column selected 내용이 서로 달라서,
    별개의 루틴으로 구성함.

In [23]:
def date_set(datei): # 하루 하루 데이터를 받아야 함.
# end_date를 먼저 입력하고 start date 입력. 반대로 하면 start date가 이전날짜로  reset되어짐
    driver.find_element(By.ID, 'endDd').clear()
    driver.find_element(By.ID, 'endDd').send_keys(datei)
    time.sleep(1)
    driver.find_element(By.ID, 'strtDd').clear()
    driver.find_element(By.ID, 'strtDd').send_keys(datei)
    time.sleep(1)
    return

In [24]:
def push_button_1(): # 조회 button push
    xp = '/html/body/div[2]/section[2]/section/section/div/div[2]/form/div[1]/div/a' 
    # use full xpath to avoid 'Message: element not interactable' Error
    driver.find_element(By.XPATH, xp).click()
    time.sleep(1) # 여유시간 배분
    css_sel = 'div.loading-bar-wrap.small' # 각기 다른 loading 페이지에서 공통적으로 사용됨
    element = WebDriverWait(driver, 60).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, css_sel)))
    # 위 라인은 pop up 창이 사라질 때까지 기다리게 해 줌
    time.sleep(1) # 여유시간 배분
    return

In [25]:
base_data_directory = './data/base_data/stock_market_holydays/'
opening_days_kor = pd.read_pickle(base_data_directory+'opening_days_kor.pkl') # 한국 개장일 데이터 
def is_opening_day(date): # 개장일 확인
    date = datetime.datetime.strptime(date, "%Y%m%d").date()
    if date in list(opening_days_kor):
        return True
    else:
        return False

In [26]:
index_name = ['investor', 'sell_quantity', 'buy_quantity', 'pure_buy_quantity', 'sell', 'buy', 'pure_buy']
column_name = ['financial', 'insurance', 'invtrust', 'privequity', 'bank', 'financeetc', 'pension',
              'institution', 'corporateetc', 'retail', 'foreigner', 'foreigneretc', 'total' ]
def get_daily_data(date_range):
    df_org = None
    for datei in date_range: 
        
        if not is_opening_day(datei): # 나중(5월 2일 이후)에 개장일만 수집하도록 수정할 것
            continue

        date_set(datei)
        push_button_1()
        # df = pd.read_html(driver.page_source, 
        #                   attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]
        
        df = pd.read_html(io.StringIO(str(driver.page_source)), 
                          attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]
        df.columns = index_name
        df_new = df[['investor', 'pure_buy']] # 순매수 금액
        df_new.set_index('investor', inplace=True)
        dft = df_new.T
        dft.columns = column_name
        dft.insert(0, "date", datetime.datetime.strptime(datei, "%Y%m%d"))
        dft.reset_index(drop=True, inplace=True)
        if df_org is None:
            df_org = dft.copy()
            continue
#         df_org = df_org.append(dft, ignore_index=True) # append will be depreciated
        df_org =pd.concat([df_org,dft], ignore_index=True)
        
    return df_org

In [27]:
def get_data_company_investor(com_name, start_date, end_date):
    
    com_ticker = com_name[:6]
    # 회사이름 입력 Q 버튼
    driver.find_element(By.CSS_SELECTOR, '#btnisuCd_finder_stkisu0_1').click()
    time.sleep(2)

    # pop up된 입력창에서 회사이름 입력
    id_name = 'searchText__finder_stkisu0_1'
    element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.ID, id_name)))
    driver.find_element(By.ID, id_name).clear()
    time.sleep(1)
    
    driver.find_element(By.ID, 'searchText__finder_stkisu0_1').send_keys(com_name)
    time.sleep(1)

    # 검색 버튼 푸시
    css_name = '#searchBtn__finder_stkisu0_1'
    element = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css_name)))
    driver.find_element(By.CSS_SELECTOR, css_name).click()
    time.sleep(1)

    # 테이블에서 최종 선택
    css_sel = '#jsGrid__finder_stkisu0_1 > tbody > tr:nth-child(1) > td:nth-child(1)'

    element = WebDriverWait(driver, 60).until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, css_sel), com_ticker))
    driver.find_element(By.CSS_SELECTOR, css_sel).click()
    time.sleep(2)
    
    date_range = convert_date(start_date, end_date)
    return get_daily_data(date_range)

In [28]:
def convert_date(start_date, end_date):
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    date_range = []
    for x in date_range_ts:
        date_range.append(datetime.datetime.strftime(x, "%Y%m%d"))
    return date_range

In [29]:
# 투자자별 URL로 변경
main_url = 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020203'
driver.get(main_url)
time.sleep(2)

In [30]:
# Message: element not interactable Error 방지용. 클릭하기 위하여는 그 위치가 클릭할 수 있게 노출되어 있어야 함
# 투자자별 거래실적 버튼이 위치한 곳으로 화면 scroll 

# id가 jsOpenView_1 인 element 를 찾음
stop_tag = driver.find_element(By.ID, 'jsOpenView_1')

# jsOpenView_1 element 까지 스크롤
action = ActionChains(driver)
action.move_to_element(stop_tag).perform()

In [31]:
# 투자자별 거래 실적 버튼 클릭
driver.find_element(By.ID, 'jsOpenView_1').click()
time.sleep(2)

In [32]:
# 백만원 단위 표시 선정
css_sel = '#MDCSTAT023_FORM > div.CI-MDI-UNIT-WRAP > div > p:nth-child(2) > select.CI-MDI-UNIT-MONEY > option:nth-child(3)'

driver.find_element(By.CSS_SELECTOR, css_sel).click()
time.sleep(1)

In [33]:
# code = { '011070': ['LG이노텍', 'lginnotek'],
#          '028050': ['삼성엔지니어링', 'ssengineering'],
#          '361610': ['SK아이이테크놀로지', 'skietech'],
#          '086280': ['현대글로비스', 'glovis'],
#          '302440': ['SK바이오사이언스', 'skbio']}

In [34]:
pkl_directory = 'data/company_pkl/investors_data/'
modi_directory = 'data/company_pkl/'
# modification_time = {} # 데이터가 생성된 시간 저장 dictionary
modification_time = pd.read_pickle(modi_directory + 'modification_time_company_inv.pkl')
pkl_name= 'sec_investors.pkl'
# pkl_name= 'lginnotek_investors.pkl'
# pkl_name= 'skbio_investors.pkl'

In [35]:
df_o = pd.read_pickle(pkl_directory + pkl_name)
df_o.tail()

,date,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc
1232,2025-01-03,-204685.0,35366.0,116408.0,81818.0,7114.0,17935.0,6633.0,381.0,1930.0,598.0,53677.0,-766.0
1233,2025-01-06,-202253.0,95839.0,52991.0,-16806.0,19444.0,23809.0,11149.0,534.0,13093.0,1768.0,54320.0,-898.0
1234,2025-01-07,4123.0,-48900.0,-13323.0,-76475.0,14847.0,43327.0,1621.0,160.0,2124.0,1074.0,58531.0,-430.0
1235,2025-01-08,-337492.0,275671.0,23450.0,-69314.0,28721.0,48756.0,5613.0,699.0,6583.0,2393.0,40694.0,-2324.0
1236,2025-01-09,98401.0,43374.0,-183639.0,-185534.0,13414.0,12515.0,-25561.0,209.0,2217.0,-898.0,41642.0,222.0


In [36]:
start_date = df_o['date'].iloc[len(df_o)-1]
start_date

Timestamp('2025-01-09 00:00:00')

In [37]:
pkl_directory = 'data/company_pkl/investors_data/'
modi_directory = 'data/company_pkl/'
# modification_time = {} # 데이터가 생성된 시간 저장 dictionary
modification_time = pd.read_pickle(modi_directory + 'modification_time_company_inv.pkl')
total = len(code)

for i, (key, val) in enumerate(code.items()):
    com_name = "/".join([key, val[0]])
    pkl_name= '{}_investors.pkl'.format(val[1])
    try :
        df_o = pd.read_pickle(pkl_directory + pkl_name)
        start_date = df_o['date'].iloc[len(df_o)-1]
    except :
        start_date = datetime.date(2022, 1, 1)   # 데이터 취득 시작 일자 
        
#     start_date = datetime.date(2023, 5, 11) # 데이터 취득 오류시 일시 사용

    end_date = datetime.date.today()
    df_out = get_data_company_investor(com_name, start_date, end_date)
    try :
        df_out = df_out[df_o.columns]     
        df_o = concat_df(df_o, df_out) # append df to original df
    except :
        df_col = ['date', 'retail', 'foreigner', 'institution', 'financial', 'invtrust',
                  'pension', 'privequity', 'bank', 'insurance', 'financeetc',
                  'corporateetc', 'foreigneretc']
        df_out = df_out[df_col]
        df_o = df_out.copy()
        
    # 개장일이 아닌 row는 삭제
    df_o = select_openingdays(df_o, opening_days_kor)
    
    df_o.to_pickle(pkl_directory+pkl_name)
    df_o.to_csv(pkl_directory+pkl_name.replace('pkl','csv'))
    # modification_time.loc[pkl_name][0] = datetime.datetime.now()
    modification_time.loc[0, pkl_name] = datetime.datetime.now()
    
    print(com_name, f'{i+1}/{total}', end=', ') # 진행상황 확인용
    
modification_time.to_pickle(modi_directory+'modification_time_company_inv.pkl')
modification_time.to_csv(modi_directory+'modification_time_company_inv.csv')

005930/삼성전자 1/52, 373220/LG에너지솔루션 2/52, 000660/SK하이닉스 3/52, 207940/삼성바이오로직스 4/52, 006400/삼성SDI 5/52, 051910/LG화학 6/52, 005935/삼성전자우 7/52, 005380/현대차 8/52, 035420/NAVER 9/52, 000270/기아 10/52, 035720/카카오 11/52, 005490/POSCO홀딩스 12/52, 105560/KB금융 13/52, 028260/삼성물산 14/52, 068270/셀트리온 15/52, 012330/현대모비스 16/52, 055550/신한지주 17/52, 066570/LG전자 18/52, 003670/포스코퓨처엠 19/52, 096770/SK이노베이션 20/52, 033780/KT&G 21/52, 030200/KT 22/52, 003550/LG 23/52, 034730/SK 24/52, 032830/삼성생명 25/52, 086790/하나금융지주 26/52, 009150/삼성전기 27/52, 015760/한국전력 28/52, 034020/두산에너빌리티 29/52, 010130/고려아연 30/52, 017670/SK텔레콤 31/52, 011200/HMM 32/52, 000810/삼성화재 33/52, 051900/LG생활건강 34/52, 010950/S-Oil 35/52, 259960/크래프톤 36/52, 018260/삼성에스디에스 37/52, 329180/현대중공업 38/52, 003490/대한항공 39/52, 036570/엔씨소프트 40/52, 009830/한화솔루션 41/52, 316140/우리금융지주 42/52, 090430/아모레퍼시픽 43/52, 011170/롯데케미칼 44/52, 024110/기업은행 45/52, 138040/메리츠금융지주 46/52, 377300/카카오페이 47/52, 011070/LG이노텍 48/52, 028050/삼성엔지니어링 49/52, 361610/SK아이이테크놀로지 50/52, 086280/현대글로비스

In [38]:
driver.close()
driver.quit()

In [ ]:
code

In [ ]:
code =  { 
 '011070': ['LG이노텍', 'lginnotek'],
 '028050': ['삼성엔지니어링', 'ssengineering'],
 '361610': ['SK아이이테크놀로지', 'skietech'],
 '086280': ['현대글로비스', 'glovis'],
 '302440': ['SK바이오사이언스', 'skbio']}

In [ ]:
code

In [ ]:
code

In [ ]:
code ={
 '329180': ['현대중공업', 'hhi'],
 '003490': ['대한항공', 'koreanair'],
 '036570': ['엔씨소프트', 'ncsoft'],
 '009830': ['한화솔루션', 'hanhwasol'],
 '316140': ['우리금융지주', 'woorifg'],
 '090430': ['아모레퍼시픽', 'amore'],
 '011170': ['롯데케미칼', 'lottechem'],
 '024110': ['기업은행', 'ibk'],
 '138040': ['메리츠금융지주', 'meritz'],
 '377300': ['카카오페이', 'kakaopay'],
 '011070': ['LG이노텍', 'lginnotek'],
 '028050': ['삼성엔지니어링', 'ssengineering'],
 '361610': ['SK아이이테크놀로지', 'skietech'],
 '086280': ['현대글로비스', 'glovis'],
 '302440': ['SK바이오사이언스', 'skbio']}

In [ ]:
code ={
 '090430': ['아모레퍼시픽', 'amore'],
 '011170': ['롯데케미칼', 'lottechem'],
 '024110': ['기업은행', 'ibk'],
 '138040': ['메리츠금융지주', 'meritz'],
 '377300': ['카카오페이', 'kakaopay'],
 '011070': ['LG이노텍', 'lginnotek'],
 '028050': ['삼성엔지니어링', 'ssengineering'],
 '361610': ['SK아이이테크놀로지', 'skietech'],
 '086280': ['현대글로비스', 'glovis'],
 '302440': ['SK바이오사이언스', 'skbio']}

In [ ]:
input("aa")

In [ ]:
k = int(input("aa"))
k + 5